In [1]:
%matplotlib inline
import cv2
import numpy as np
import time

In [2]:
person_net = cv2.dnn.readNet("person-detection-retail-0013.bin",
                             "person-detection-retail-0013.xml")

In [3]:
cap = cv2.VideoCapture(0)

width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = 25
font = cv2.FONT_HERSHEY_SIMPLEX

# Define the codec and create VideoWriter object.The output is stored in 'outpy.avi' file.
writer = cv2.VideoWriter('PersonDetection.mp4', cv2.VideoWriter_fourcc(*'DIVX'), fps, (width, height))

while cv2.waitKey(1) < 0:
    ret, frame = cap.read()
    if not ret:
        break
 
    try:
        start_time = time.time()
        
        blob = cv2.dnn.blobFromImage(frame,size = (200,200))
        person_net.setInput(blob)
        out = person_net.forward()


        for detection in out.reshape(-1, 7):
            confidence = float(detection[2])
            xmin = int(detection[3] * frame.shape[1])
            ymin = int(detection[4] * frame.shape[0])
            xmax = int(detection[5] * frame.shape[1])
            ymax = int(detection[6] * frame.shape[0])

            if confidence > 0.4:
                cv2.rectangle(frame, (xmin, ymin), (xmax, ymax), color=(0, 255, 0),thickness = 5)
                


        
        end_time = time.time()
        fps = np.round(1/(end_time-start_time))
        cv2.putText(frame,text="Current FPS: {}".format(fps),
        org=(0,30),  # X. Y
        fontFace=font,fontScale= 0.5,color=(0,0,255),thickness=2,lineType=cv2.LINE_AA)
        
        writer.write(frame)
        cv2.imshow('OpenVINO face detection', frame)
        

    except:
        print("Error")
        pass
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break 
        

    
# When everything done, realease the capture and destroy the windows
cap.release()
writer.release()
cv2.destroyAllWindows()